## Make nested regioinal model figure for JOSS paper

In [1]:
import sys
sys.path.append("/home/149/ab8992/tasman-tides/")
sys.path.append("/home/149/ab8992/cosima_regional/regional-mom6")
import regional_mom6 as rmom6
import xarray as xr
import ttidelib as tt
import scipy
import cmocean
import os
from pathlib import Path
cmap = cmocean.cm.dense_r
import matplotlib
import matplotlib.pyplot as plt
earth_cmap = matplotlib.colormaps["gist_earth"]
from datetime import timedelta
# import filtering
import numpy as np
import dask
from dask.distributed import Client,default_client
import xrft
import eofs

client = tt.startdask()



def plot_surfacespeed(data,**kwargs):

    cmap = cmocean.cm.dense_r
    earth_cmap = matplotlib.cm.get_cmap("gist_earth")
    fig,ax = plt.subplots(1,figsize = (15,12))
    # Set the background colour to the plot to the lowest value in the cmap
    ax.set_facecolor(cmap(0))
    
    data["speed"].plot(vmax = 4,vmin = 0,ax = ax,cmap = cmap,add_colorbar = False)
    data["bathy"].plot(cmap = earth_cmap,vmin = -1000,vmax = 1500,ax = ax,add_colorbar = False)
    ax.set_title("Surface Speed")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.text(0.95, 0.95, f"Day {int(data.time.values//24)}", transform=ax.transAxes, fontsize=15, va="top", ha="right",color = "linen")
    return fig



client


<Client: 'tcp://127.0.0.1:38797' processes=7 threads=28, memory=112.00 GiB>


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 112.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38797,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 112.00 GiB
Comm: tcp://127.0.0.1:42525,Total threads: 4
Dashboard: /proxy/37889/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:43719,


## Interpolate bathymetry

In [4]:

xextent = np.array([130.0,190.0])
yextent = np.array([-65.0,-15.0])
yextent = [-56,-26]
xextent = [142,180]

expt = rmom6.experiment(
    longitude_extent = xextent,
    latitude_extent = yextent,
    date_range = ["2015-01-01 00:00:00", "2015-12-31 00:00:00"],
    resolution = 0.08,
    number_vertical_layers = 100,
    layer_thickness_ratio = 10,
    depth = 6000,
    mom_run_dir = "/g/data/nm03/ab8992/glorys-download/for_joss_figure",
    mom_input_dir = "/g/data/nm03/ab8992/glorys-download/for_joss_figure",
    toolpath_dir = "/home/157/ahg157/repos/mom5/src/tools/",
)

expt.setup_bathymetry(
    bathymetry_path = '/g/data/v45/ab8992/synbath2/SYNBATH_V1.2.nc',
    vertical_coordinate_name = "z",
    longitude_coordinate_name = "lon",
    latitude_coordinate_name = "lat",
    # chunks = {"longitude": 500, "latitude": 500},
    )

0.3.0
Begin regridding bathymetry...

If this process hangs it means that the chosen domain might be too big to handle this way. After ensuring access to appropriate computational resources, try calling ESMF directly from a terminal in the input directory via

mpirun ESMF_Regrid -s bathymetry_original.nc -d bathymetry_unfinished.nc -m bilinear --src_var elevation --dst_var elevation --netcdf4 --src_regional --dst_regional

For details see https://xesmf.readthedocs.io/en/latest/large_problems_on_HPC.html

Afterwards, we run 'tidy_bathymetry' method to skip the expensive interpolation step, and finishing metadata, encoding and cleanup.
Regridding in parallel: True


: 

In [4]:
experiment = "full-80"

resolution = experiment.split("-")[-1]

speed = xr.open_mfdataset(f"/g/data/nm03/ab8992/outputs/{experiment}/**/surface.nc",decode_times=False,chunks = {"time":1}).speed
speed_glorys = xr.open_mfdataset("/g/data/nm03/ab8992/glorys-download/for_joss_figure/ic_for_figure.nc")
speed_glorys = (speed_glorys.uo**2 + speed_glorys.vo**2).rename("speed")**0.5
# interpolate the xq and yq onto xh and yh using xarray built in method
bathy = xr.open_mfdataset(f"/g/data/nm03/ab8992/ttide-inputs/full-{resolution}/topog_raw.nc",decode_times = False).elevation
bathy = bathy.rename({"lat":"yh","lon":"xh"})
bathy = bathy.where(bathy > 0).persist()
bathy20 = xr.open_mfdataset(f"/g/data/nm03/ab8992/ttide-inputs/full-20/topog_raw.nc",decode_times = False).elevation
bathy20 = bathy20.rename({"lat":"yh","lon":"xh"})
bathy20 = bathy20.where(bathy > 0).persist()
data = xr.Dataset(
    {
        "speed":speed,
        "bathy":bathy
    }
)

data_glorys = xr.Dataset(
    {
        "speed":speed_glorys,
        "bathy":bathy20
    }
)

fig = plot_surfacespeed(data.isel(time = -1))

AttributeError: 'Dataset' object has no attribute 'uo'

In [6]:
xr.open_mfdataset("/g/data/nm03/ab8992/glorys-download/for_joss_figure/ic_for_figure.nc")

<xarray.Dataset>
Dimensions:    (depth: 50, latitude: 601, time: 1, longitude: 721)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 -65.0 -64.92 -64.83 ... -15.17 -15.08 -15.0
  * time       (time) datetime64[ns] 2014-02-14
  * longitude  (longitude) float32 nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    *empty*
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.1.0

In [5]:
speed_glorys

<xarray.Dataset>
Dimensions:    (depth: 50, latitude: 601, time: 1, longitude: 721)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 -65.0 -64.92 -64.83 ... -15.17 -15.08 -15.0
  * time       (time) datetime64[ns] 2014-02-14
  * longitude  (longitude) float32 nan nan nan nan nan ... nan nan nan nan nan
Data variables:
    *empty*
Attributes: (12/25)
    Conventions:               CF-1.4
    bulletin_date:             2021-07-07 00:00:00
    bulletin_type:             operational
    comment:                   CMEMS product
    domain_name:               GL12
    easting:                   longitude
    ...                        ...
    references:                http://www.mercator-ocean.fr
    source:                    MERCATOR GLORYS12V1
    title:                     daily mean fields from Global Ocean Physics An...
    z_max:                     5727.9169921875
    z_min:                     0.49402499198913574
    copernicusmarine_version:  1.1.0

In [ ]:
fig = plot_surfacespeed(data_glorys.isel(time = -1))
